# Initial Value Problem Solvers

This notebook demonstrates `torchscience.integration.initial_value_problem` - differentiable ODE solvers for PyTorch.

We'll explore three physics examples:
1. **Double Pendulum** - chaotic dynamics with adaptive stepping
2. **Lotka-Volterra** - ecology model with TensorDict state
3. **Neural ODE** - learning dynamics with adjoint gradients

Each example showcases different solver capabilities with interactive Plotly visualizations and Manim animations.

In [ ]:
import time

import numpy as np
import plotly.express as px  # noqa: F401
import plotly.graph_objects as go  # noqa: F401
import torch  # noqa: F401
from plotly.subplots import make_subplots  # noqa: F401
from tensordict import TensorDict  # noqa: F401

try:
    from manim import *  # noqa: F401, F403

    MANIM_AVAILABLE = True
except ImportError:
    MANIM_AVAILABLE = False
    print("Note: manim not installed. Animation examples will be skipped.")
from IPython.display import Video  # noqa: F401

from torchscience.integration.initial_value_problem import (  # noqa: F401
    adjoint,
    backward_euler,
    dormand_prince_5,
    euler,
    midpoint,
    runge_kutta_4,
)

In [ ]:
# Configuration
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Plotly defaults for dark theme
plotly_template = "plotly_dark"  # noqa: F841

## 1. Double Pendulum: Chaos and Adaptive Stepping

The double pendulum is a classic chaotic system - two rigid rods connected at joints, swinging under gravity. Small changes in initial conditions lead to dramatically different trajectories.

**State vector:** `[theta1, theta2, omega1, omega2]`
- `theta1`, `theta2`: angles from vertical
- `omega1`, `omega2`: angular velocities

**Why it's a good test case:**
- Chaotic dynamics require adaptive step sizes
- Large state changes stress error estimation
- Visually dramatic for demonstrations

In [ ]:
def double_pendulum_dynamics(t, state):
    """
    Double pendulum equations of motion.

    Parameters:
        t: time (unused, system is autonomous)
        state: [theta1, theta2, omega1, omega2]

    Returns:
        [dtheta1/dt, dtheta2/dt, domega1/dt, domega2/dt]
    """
    # Physical parameters
    g = 9.81  # gravity
    L1 = 1.0  # length of rod 1
    L2 = 1.0  # length of rod 2
    m1 = 1.0  # mass 1
    m2 = 1.0  # mass 2

    theta1, theta2, omega1, omega2 = (
        state[..., 0],
        state[..., 1],
        state[..., 2],
        state[..., 3],
    )

    delta = theta2 - theta1

    # Denominator terms
    den1 = (m1 + m2) * L1 - m2 * L1 * torch.cos(delta) ** 2
    den2 = (L2 / L1) * den1

    # Angular accelerations
    domega1 = (
        -m2 * L1 * omega1**2 * torch.sin(delta) * torch.cos(delta)
        + m2 * g * torch.sin(theta2) * torch.cos(delta)
        + m2 * L2 * omega2**2 * torch.sin(delta)
        - (m1 + m2) * g * torch.sin(theta1)
    ) / den1

    domega2 = (
        +m2 * L2 * omega2**2 * torch.sin(delta) * torch.cos(delta)
        + (m1 + m2) * g * torch.sin(theta1) * torch.cos(delta)
        + (m1 + m2) * L1 * omega1**2 * torch.sin(delta)
        - (m1 + m2) * g * torch.sin(theta2)
    ) / den2

    return torch.stack([omega1, omega2, domega1, domega2], dim=-1)

In [ ]:
# Initial conditions: moderate angles for chaotic but tractable dynamics
# [theta1, theta2, omega1, omega2]
# Starting at ~120 degrees (2.1 rad) shows chaotic behavior without
# the extreme step requirements of nearly inverted pendulums.
y0 = torch.tensor([2.1, 2.5, 0.5, 0.0], dtype=torch.float64, device=device)

# Solve with dormand_prince_5
start = time.perf_counter()
y_final, interp = dormand_prince_5(
    double_pendulum_dynamics,
    y0,
    t_span=(0.0, 15.0),
    rtol=1e-6,
    atol=1e-8,
    max_steps=50000,
)
elapsed = time.perf_counter() - start

print(f"Integration completed in {elapsed:.3f}s")
print(f"Number of steps: {interp.n_steps}")
print(f"Final state: theta1={y_final[0]:.3f}, theta2={y_final[1]:.3f}")

In [ ]:
# Sample dense trajectory using interpolant
t_dense = torch.linspace(0, 15, 1500, dtype=torch.float64, device=device)
trajectory = interp(t_dense)

theta1 = trajectory[:, 0].cpu().numpy()
theta2 = trajectory[:, 1].cpu().numpy()
omega1 = trajectory[:, 2].cpu().numpy()
omega2 = trajectory[:, 3].cpu().numpy()
t_np = t_dense.cpu().numpy()

In [ ]:
# 3D trajectory: (theta1, theta2, t) colored by omega1
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=theta1,
            y=theta2,
            z=t_np,
            mode="lines",
            line=dict(
                color=omega1,
                colorscale="Viridis",
                width=2,
                colorbar=dict(title="omega1"),
            ),
            name="Trajectory",
        )
    ]
)

fig.update_layout(
    title="Double Pendulum: Chaotic Trajectory",
    scene=dict(
        xaxis_title="theta1 (rad)",
        yaxis_title="theta2 (rad)",
        zaxis_title="time (s)",
    ),
    template=plotly_template,
    width=800,
    height=600,
)
fig.show()

In [ ]:
# Phase portrait: (theta1, omega1) colored by time
# Use markers mode for color mapping along the trajectory
fig = go.Figure(
    data=[
        go.Scatter(
            x=theta1,
            y=omega1,
            mode="markers",
            marker=dict(
                color=t_np,
                colorscale="Plasma",
                size=2,
                colorbar=dict(title="time (s)"),
            ),
        )
    ]
)

fig.update_layout(
    title="Phase Portrait: theta1 vs omega1",
    xaxis_title="theta1 (rad)",
    yaxis_title="omega1 (rad/s)",
    template=plotly_template,
    width=700,
    height=500,
)
fig.show()

In [ ]:
# Convert angles to Cartesian coordinates
L1, L2 = 1.0, 1.0

x1 = L1 * np.sin(theta1)
y1 = -L1 * np.cos(theta1)
x2 = x1 + L2 * np.sin(theta2)
y2 = y1 - L2 * np.cos(theta2)

print(
    f"Mass 1 range: x=[{x1.min():.2f}, {x1.max():.2f}], y=[{y1.min():.2f}, {y1.max():.2f}]"
)
print(
    f"Mass 2 range: x=[{x2.min():.2f}, {x2.max():.2f}], y=[{y2.min():.2f}, {y2.max():.2f}]"
)

In [ ]:
# Double pendulum animation (requires manim)
if MANIM_AVAILABLE:
    from manim import (
        BLUE,
        ORIGIN,
        RED,
        WHITE,
        YELLOW,
        Dot,
        Line,
        Scene,
        TracedPath,
    )

    class DoublePendulumScene(Scene):
        def construct(self):
            # Physical parameters
            L1, L2 = 2.0, 2.0  # Scaled for visibility

            # Get trajectory data (use global variables from previous cells)
            # Sample every 10th point for smoother animation
            theta1_anim = theta1[::10]
            theta2_anim = theta2[::10]

            # Create pendulum components
            pivot = Dot(ORIGIN, color=WHITE)

            def get_positions(i):
                t1, t2 = theta1_anim[i], theta2_anim[i]
                p1 = np.array([L1 * np.sin(t1), -L1 * np.cos(t1), 0])
                p2 = p1 + np.array([L2 * np.sin(t2), -L2 * np.cos(t2), 0])
                return p1, p2

            p1_0, p2_0 = get_positions(0)

            rod1 = Line(ORIGIN, p1_0, color=BLUE, stroke_width=4)
            rod2 = Line(p1_0, p2_0, color=RED, stroke_width=4)
            mass1 = Dot(p1_0, color=BLUE, radius=0.15)
            mass2 = Dot(p2_0, color=RED, radius=0.15)

            # Trail for mass2
            trail = TracedPath(
                mass2.get_center,
                stroke_color=YELLOW,
                stroke_width=1,
                stroke_opacity=0.5,
            )

            self.add(pivot, rod1, rod2, mass1, mass2, trail)

            # This defines the scene - to render, use: manim -pql notebook.py DoublePendulumScene
            # Or in a Jupyter cell with manim kernel: %%manim -qm DoublePendulumScene
            self.wait(2)
else:
    print("Manim not available - skipping animation cell")

### Observations

- **Adaptive Stepping:** Notice how `dormand_prince_5` takes many small steps during rapid changes and larger steps during calmer periods.
- **Step Count:** The solver efficiently handles the chaotic dynamics with automatic step size adaptation.
- **Sensitivity:** Try changing initial conditions by 0.01 rad - the trajectories diverge dramatically within seconds.
- **Dense Output:** The interpolant provides smooth trajectories at any time point without re-solving.

## 2. Lotka-Volterra: TensorDict and Solver Comparison

The Lotka-Volterra equations model predator-prey dynamics in ecology. Populations oscillate in closed orbits - when prey is abundant, predators thrive; as prey declines, predators starve.

**Equations:**
- `dx/dt = alpha*x - beta*x*y` (prey growth minus predation)
- `dy/dt = delta*x*y - gamma*y` (predator growth minus death)

**Why it's a good test case:**
- Simple, intuitive dynamics
- Closed orbits test energy conservation
- TensorDict state demonstrates structured data handling
- Smooth enough for fixed-step solvers

In [ ]:
def lotka_volterra(t, state):
    """
    Lotka-Volterra predator-prey dynamics using TensorDict.

    Parameters:
        t: time (unused, system is autonomous)
        state: TensorDict with 'prey' and 'predator' keys

    Returns:
        TensorDict with derivatives
    """
    # Parameters
    alpha = 1.0  # prey growth rate
    beta = 0.1  # predation rate
    delta = 0.075  # predator growth rate
    gamma = 1.5  # predator death rate

    x = state["prey"]  # prey population
    y = state["predator"]  # predator population

    dx = alpha * x - beta * x * y
    dy = delta * x * y - gamma * y

    return TensorDict(
        {
            "prey": dx,
            "predator": dy,
        },
        batch_size=state.batch_size,
    )

In [ ]:
# Initial populations
state0 = TensorDict(
    {
        "prey": torch.tensor([40.0], device=device),
        "predator": torch.tensor([9.0], device=device),
    },
    batch_size=[],
)

# Solve with runge_kutta_4 (fixed step, good for smooth problems)
state_final, interp_lv = runge_kutta_4(
    lotka_volterra,
    state0,
    t_span=(0.0, 50.0),
    dt=0.01,
)

print(
    f"Initial: prey={state0['prey'].item():.1f}, predator={state0['predator'].item():.1f}"
)
print(
    f"Final:   prey={state_final['prey'].item():.1f}, predator={state_final['predator'].item():.1f}"
)

In [ ]:
# Compare solvers on Lotka-Volterra
solvers = {
    "euler": (euler, {"dt": 0.01}),
    "midpoint": (midpoint, {"dt": 0.05}),
    "runge_kutta_4": (runge_kutta_4, {"dt": 0.1}),
    "dormand_prince_5": (dormand_prince_5, {}),
}

# Reference solution with tight tolerances
state_ref, _ = dormand_prince_5(
    lotka_volterra, state0, t_span=(0.0, 50.0), rtol=1e-6, atol=1e-8
)

results = {}
interpolants = {}  # Store interpolants for cell 16
for name, (solver, kwargs) in solvers.items():
    start = time.perf_counter()
    state_final, interp = solver(
        lotka_volterra, state0, t_span=(0.0, 50.0), **kwargs
    )
    elapsed = time.perf_counter() - start

    error = torch.sqrt(
        (state_final["prey"] - state_ref["prey"]) ** 2
        + (state_final["predator"] - state_ref["predator"]) ** 2
    ).item()

    results[name] = {
        "time": elapsed,
        "error": error,
        "prey_final": state_final["prey"].item(),
        "predator_final": state_final["predator"].item(),
    }
    interpolants[name] = interp  # Store interpolant

# Display comparison
print(
    f"{'Solver':<20} {'Time (s)':<12} {'Error':<12} {'Prey Final':<12} {'Predator Final':<12}"
)
print("-" * 68)
for name, res in results.items():
    print(
        f"{name:<20} {res['time']:<12.4f} {res['error']:<12.6f} {res['prey_final']:<12.2f} {res['predator_final']:<12.2f}"
    )

In [ ]:
# Time series comparison
t_eval = torch.linspace(0, 50, 500, device=device)

fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    subplot_titles=("Prey Population", "Predator Population"),
)

colors = {
    "euler": "red",
    "midpoint": "orange",
    "runge_kutta_4": "green",
    "dormand_prince_5": "blue",
}

for name in solvers.keys():
    interp = interpolants[name]  # Reuse stored interpolant

    # Batched query - much faster
    states = interp(t_eval)
    # Handle both TensorDict and tensor return types
    if hasattr(states, "keys"):  # TensorDict
        prey_traj = states["prey"].squeeze().cpu().numpy()
        pred_traj = states["predator"].squeeze().cpu().numpy()
    else:
        prey_traj = states[..., 0].cpu().numpy()
        pred_traj = states[..., 1].cpu().numpy()

    fig.add_trace(
        go.Scatter(
            x=t_eval.cpu().numpy(),
            y=prey_traj,
            name=name,
            line=dict(color=colors[name]),
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=t_eval.cpu().numpy(),
            y=pred_traj,
            name=name,
            line=dict(color=colors[name]),
            showlegend=False,
        ),
        row=2,
        col=1,
    )

fig.update_layout(
    title="Solver Comparison: Lotka-Volterra",
    template=plotly_template,
    height=600,
)
fig.update_xaxes(title_text="Time", row=2, col=1)
fig.update_yaxes(title_text="Prey", row=1, col=1)
fig.update_yaxes(title_text="Predator", row=2, col=1)
fig.show()

In [ ]:
# Phase portrait with multiple initial conditions
fig = go.Figure()

# Multiple initial conditions
initial_conditions = [
    (40.0, 9.0),
    (50.0, 5.0),
    (30.0, 12.0),
    (60.0, 8.0),
]

t_eval_phase = torch.linspace(0, 30, 300, device=device)

for x0, y0 in initial_conditions:
    state_ic = TensorDict(
        {
            "prey": torch.tensor([x0], device=device),
            "predator": torch.tensor([y0], device=device),
        },
        batch_size=[],
    )

    _, interp = runge_kutta_4(
        lotka_volterra, state_ic, t_span=(0.0, 30.0), dt=0.05
    )

    # Batched query for efficiency
    states = interp(t_eval_phase)
    if hasattr(states, "keys"):  # TensorDict
        prey_traj = states["prey"].squeeze().cpu().numpy()
        pred_traj = states["predator"].squeeze().cpu().numpy()
    else:
        prey_traj = states[..., 0].cpu().numpy()
        pred_traj = states[..., 1].cpu().numpy()

    fig.add_trace(
        go.Scatter(
            x=prey_traj,
            y=pred_traj,
            mode="lines",
            name=f"IC: ({x0}, {y0})",
        )
    )

    # Mark initial point
    fig.add_trace(
        go.Scatter(
            x=[x0],
            y=[y0],
            mode="markers",
            marker=dict(size=10, symbol="circle"),
            showlegend=False,
        )
    )

fig.update_layout(
    title="Lotka-Volterra Phase Portrait",
    xaxis_title="Prey Population",
    yaxis_title="Predator Population",
    template=plotly_template,
    width=700,
    height=600,
)
fig.show()

In [ ]:
# Lotka-Volterra animation (requires manim)
if MANIM_AVAILABLE:
    from manim import (
        DOWN,
        GREEN,
        LEFT,
        RED,
        RIGHT,
        YELLOW,
        Axes,
        Dot,
        Rectangle,
        Scene,
        Text,
        VGroup,
    )

    class LotkaVolterraScene(Scene):
        """
        Static scene template demonstrating Lotka-Volterra visualization layout.

        This scene shows the basic structure. For full animation:
        1. Extract this class to a .py file
        2. Import trajectory data (prey_traj, pred_traj from cell 17)
        3. Use self.play() with UpdateFromAlphaFunc to animate bars and dot

        Render with: manim -pql script.py LotkaVolterraScene
        """

        def construct(self):
            # Two-panel layout: populations on left, phase portrait on right

            # Left panel: population bars (scaled to approximate population values)
            left_panel = VGroup()
            # Scale: width = population / 20 for visibility
            prey_bar = Rectangle(
                height=0.3, width=2.0, color=GREEN, fill_opacity=0.8
            )  # prey ~ 40
            pred_bar = Rectangle(
                height=0.3, width=0.45, color=RED, fill_opacity=0.8
            )  # predator ~ 9
            prey_label = Text("Prey: 40", font_size=18).next_to(prey_bar, LEFT)
            pred_label = Text("Pred: 9", font_size=18).next_to(pred_bar, LEFT)

            left_panel.add(
                VGroup(prey_label, prey_bar).arrange(RIGHT, buff=0.3)
            )
            left_panel.add(
                VGroup(pred_label, pred_bar).arrange(RIGHT, buff=0.3)
            )
            left_panel.arrange(DOWN, buff=0.5)
            left_panel.to_edge(LEFT, buff=0.5)

            # Right panel: phase portrait axes
            axes = Axes(
                x_range=[0, 80, 20],
                y_range=[0, 30, 10],
                x_length=5,
                y_length=4,
                tips=False,
                axis_config={"include_numbers": True},
            ).to_edge(RIGHT, buff=0.5)

            axes_labels = axes.get_axis_labels(
                x_label="Prey", y_label="Predator"
            )

            self.add(left_panel, axes, axes_labels)

            # Initial point on phase portrait
            dot = Dot(axes.c2p(40, 9), color=YELLOW, radius=0.1)
            self.add(dot)

            # Static display - to animate, use UpdateFromAlphaFunc
            self.wait(2)
else:
    print("Manim not available - skipping Lotka-Volterra animation cell")

### Observations

- **TensorDict State:** Using named fields (`prey`, `predator`) makes the code self-documenting and prevents indexing errors.
- **Closed Orbits:** The phase portraits show closed curves - a conserved quantity (related to the Hamiltonian) is preserved.
- **Solver Selection:** For smooth problems like this, `runge_kutta_4` with moderate step size is efficient. Adaptive `dormand_prince_5` works but adds overhead.
- **Error Accumulation:** Lower-order methods (Euler) accumulate phase error over long integrations.

## 3. Neural ODE: Differentiable Dynamics and Adjoint Method

Neural ODEs replace discrete layers with continuous transformations. Instead of:
```
h1 = f1(h0)
h2 = f2(h1)
...
```

We have:
```
h(t) = h(0) + integral_0^T f(h(t), t; theta) dt
```

**Key benefits:**
- Continuous-depth models
- Memory-efficient gradients via adjoint method
- Invertible by design (for normalizing flows)

**This example:** Classify points from two interleaved spirals using learned continuous dynamics.

**Important insight:** In 2D, ODE trajectories cannot cross (uniqueness theorem), which limits expressiveness for non-linearly separable data. We address this using an **augmented state space** - lifting the 2D input to a higher-dimensional space where the dynamics can separate the classes.

In [ ]:
def generate_spirals(n_points=200, noise=0.1):
    """Generate two interleaved spiral datasets."""
    n = n_points // 2

    # Spiral 1
    theta1 = torch.linspace(0, 3 * np.pi, n)
    r1 = theta1 / (3 * np.pi)
    x1 = r1 * torch.cos(theta1) + noise * torch.randn(n)
    y1 = r1 * torch.sin(theta1) + noise * torch.randn(n)

    # Spiral 2 (rotated by pi)
    theta2 = torch.linspace(0, 3 * np.pi, n)
    r2 = theta2 / (3 * np.pi)
    x2 = r2 * torch.cos(theta2 + np.pi) + noise * torch.randn(n)
    y2 = r2 * torch.sin(theta2 + np.pi) + noise * torch.randn(n)

    X = torch.stack([torch.cat([x1, x2]), torch.cat([y1, y2])], dim=1).to(
        device
    )

    y = torch.cat([torch.zeros(n), torch.ones(n)]).long().to(device)

    return X, y


X, y = generate_spirals(400, noise=0.05)
print(f"Data shape: {X.shape}, Labels shape: {y.shape}")

In [ ]:
# Visualize spiral data
fig = go.Figure()

for label, name, color in [(0, "Spiral 1", "blue"), (1, "Spiral 2", "red")]:
    mask = y == label
    fig.add_trace(
        go.Scatter(
            x=X[mask, 0].cpu().numpy(),
            y=X[mask, 1].cpu().numpy(),
            mode="markers",
            name=name,
            marker=dict(size=5, color=color, opacity=0.7),
        )
    )

fig.update_layout(
    title="Interleaved Spirals Dataset",
    xaxis_title="x",
    yaxis_title="y",
    template=plotly_template,
    width=600,
    height=600,
)
fig.show()

In [ ]:
import torch.nn as nn


class ODEFunc(nn.Module):
    """Augmented state dynamics dh/dt = f(h, t) in higher-dimensional space."""

    def __init__(self, aug_dim=4, hidden_dim=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(aug_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, aug_dim),
        )

    def forward(self, t, h):
        """Compute dh/dt. Note: t is unused (autonomous system)."""
        return self.net(h)

In [ ]:
class NeuralODE(nn.Module):
    """
    Neural ODE classifier for 2D points.

    Uses an augmented state space approach: the 2D input is first lifted
    to a higher-dimensional space where the ODE dynamics can more easily
    separate the classes (in 2D, ODE trajectories cannot cross, limiting
    expressiveness for non-linearly separable data like spirals).
    """

    def __init__(self, aug_dim=4, hidden_dim=64, t_span=(0.0, 1.0)):
        super().__init__()
        self.aug_dim = aug_dim
        # Lift 2D input to augmented state space
        self.encoder = nn.Linear(2, aug_dim)
        self.odefunc = ODEFunc(aug_dim, hidden_dim)
        self.t_span = t_span
        # Project augmented state to class logits
        self.classifier = nn.Linear(aug_dim, 2)  # 2 classes

        # Use direct backprop for training - gradients flow through ODE solver
        # (adjoint method demonstrated separately in memory comparison)
        self.solver = dormand_prince_5

    def forward(self, x):
        """
        Forward pass: encode, integrate dynamics, then classify.

        x: (batch, 2) input points
        returns: (batch, 2) class logits
        """
        # Lift to augmented state space
        h0 = self.encoder(x)

        # Integrate dynamics in augmented space
        h_final, _ = self.solver(
            self.odefunc,
            h0,
            t_span=self.t_span,
            rtol=1e-3,
            atol=1e-4,
        )

        # Classify final state
        logits = self.classifier(h_final)
        return logits


# Instantiate model
model = NeuralODE(aug_dim=4, hidden_dim=64).to(device)
print(f"Model parameters: {sum(p.numel() for p in model.parameters())}")

In [ ]:
# Training setup
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)
criterion = nn.CrossEntropyLoss()

n_epochs = 200
batch_size = 64
losses = []
accuracies = []

# Training loop
for epoch in range(n_epochs):
    model.train()

    # Shuffle data
    perm = torch.randperm(len(X))
    X_shuffled = X[perm]
    y_shuffled = y[perm]

    epoch_loss = 0.0
    n_batches = 0

    for i in range(0, len(X), batch_size):
        x_batch = X_shuffled[i : i + batch_size]
        y_batch = y_shuffled[i : i + batch_size]

        optimizer.zero_grad()
        logits = model(x_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        n_batches += 1

    # Evaluate
    model.eval()
    with torch.no_grad():
        logits = model(X)
        preds = logits.argmax(dim=1)
        acc = (preds == y).float().mean().item()

    losses.append(epoch_loss / n_batches)
    accuracies.append(acc)

    if (epoch + 1) % 25 == 0:
        print(f"Epoch {epoch + 1:3d}: loss={losses[-1]:.4f}, acc={acc:.3f}")

print(f"\nFinal accuracy: {accuracies[-1]:.3f}")

In [ ]:
# Training curves
fig = make_subplots(rows=1, cols=2, subplot_titles=("Loss", "Accuracy"))

fig.add_trace(
    go.Scatter(y=losses, mode="lines", name="Loss", line=dict(color="red")),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        y=accuracies, mode="lines", name="Accuracy", line=dict(color="green")
    ),
    row=1,
    col=2,
)

fig.update_layout(
    title="Neural ODE Training",
    template=plotly_template,
    height=400,
    showlegend=False,
)
fig.update_xaxes(title_text="Epoch", row=1, col=1)
fig.update_xaxes(title_text="Epoch", row=1, col=2)
fig.update_yaxes(title_text="Loss", row=1, col=1)
fig.update_yaxes(title_text="Accuracy", row=1, col=2)
fig.show()

In [ ]:
# Decision boundary visualization
model.eval()

# Create grid
x_min, x_max = X[:, 0].min().item() - 0.5, X[:, 0].max().item() + 0.5
y_min, y_max = X[:, 1].min().item() - 0.5, X[:, 1].max().item() + 0.5

xx = torch.linspace(x_min, x_max, 100, device=device)
yy = torch.linspace(y_min, y_max, 100, device=device)
grid_x, grid_y = torch.meshgrid(xx, yy, indexing="xy")
grid_points = torch.stack([grid_x.flatten(), grid_y.flatten()], dim=1)

with torch.no_grad():
    logits = model(grid_points)
    probs = torch.softmax(logits, dim=1)[:, 1]  # P(class 1)
    probs = probs.reshape(100, 100).cpu().numpy()

fig = go.Figure()

# Decision boundary heatmap
fig.add_trace(
    go.Heatmap(
        x=xx.cpu().numpy(),
        y=yy.cpu().numpy(),
        z=probs.T,
        colorscale="RdBu",
        opacity=0.5,
        showscale=False,
    )
)

# Data points
for label, name, color in [(0, "Spiral 1", "blue"), (1, "Spiral 2", "red")]:
    mask = y == label
    fig.add_trace(
        go.Scatter(
            x=X[mask, 0].cpu().numpy(),
            y=X[mask, 1].cpu().numpy(),
            mode="markers",
            name=name,
            marker=dict(
                size=5, color=color, line=dict(width=0.5, color="white")
            ),
        )
    )

fig.update_layout(
    title="Neural ODE Decision Boundary",
    xaxis_title="x",
    yaxis_title="y",
    template=plotly_template,
    width=700,
    height=600,
)
fig.show()

In [ ]:
# Memory comparison: adjoint vs direct backprop
import gc


def count_grad_tensors():
    """Count tensors in autograd graph (approximation)."""
    gc.collect()
    return sum(
        1
        for obj in gc.get_objects()
        if isinstance(obj, torch.Tensor) and obj.requires_grad
    )


# Direct backprop
model_direct = NeuralODE(aug_dim=4, hidden_dim=64).to(device)
model_direct.solver = dormand_prince_5  # No adjoint wrapper

# Adjoint
model_adjoint = NeuralODE(aug_dim=4, hidden_dim=64).to(device)
model_adjoint.solver = adjoint(dormand_prince_5)

# Set models to eval mode for consistent comparison
model_direct.eval()
model_adjoint.eval()

x_test = X[:32]  # Small batch

print("Memory comparison (gradient tensors created):")
print("-" * 40)

# Direct
gc.collect()
base_count = count_grad_tensors()
logits = model_direct(x_test)
loss = logits.sum()
loss.backward()
direct_count = count_grad_tensors() - base_count
print(f"Direct backprop: ~{direct_count} tensors")

# Clear gradients before next comparison
model_direct.zero_grad()

# Adjoint
gc.collect()
base_count = count_grad_tensors()
logits = model_adjoint(x_test)
loss = logits.sum()
loss.backward()
adjoint_count = count_grad_tensors() - base_count
print(f"Adjoint method:  ~{adjoint_count} tensors")

### Key Takeaways

- **Adjoint Method:** Uses O(1) memory for autograd graph vs O(n_steps) for direct backprop. Essential for long integrations or large state dimensions.
- **When to Use Adjoint:**
  - Long integration times (100+ steps)
  - Large state dimension
  - Memory-constrained environments
- **When NOT to Use Adjoint:**
  - Short integrations (overhead not worth it)
  - Need exact discretization gradients
  - Differentiating through the interpolant

## Summary: Solver Selection Guide

| Solver | Order | Adaptive | Stiff | Best For |
|--------|-------|----------|-------|----------|
| `euler` | 1 | No | No | Education, quick prototypes |
| `midpoint` | 2 | No | No | Smooth problems, better than Euler |
| `runge_kutta_4` | 4 | No | No | Workhorse for non-stiff problems |
| `dormand_prince_5` | 5(4) | Yes | No | Production-quality, error control |
| `backward_euler` | 1 | No | Yes | Stiff problems (chemical kinetics, etc.) |

**Wrappers:**
- `adjoint(solver)` - Memory-efficient gradients via continuous adjoint method

## Next Steps

- Explore more examples in the torchscience documentation
- Check out normalizing flows for density estimation with Neural ODEs
- See stiff ODE examples for chemical reaction networks